In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import time

# Safari WebDriver 인스턴스 생성
driver = webdriver.Safari()


products_data = {}

try:
    # 디스코드 로그인 페이지 로드
    driver.get('https://search.danawa.com/dsearch.php?query=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&originalQuery=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&previousKeyword=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&checkedInfo=N&volumeType=allvs&page=1&limit=40&sort=priceASC&list=list&boost=true&tab=goods&addDelivery=Y&coupangMemberSort=N&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=16325265&defaultPhysicsCategoryCode=46803%7C46818%7C50379%7C0&defaultVmTab=10&defaultVaTab=387&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A')

    # 충분한 로딩 시간 대기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))
    main_window = driver.current_window_handle  # 메인 윈도우 핸들 저장
    
    # Find all products within the product list that have the class '.prod_item .searched'
    products = driver.find_elements(By.CSS_SELECTOR, ".product_list .prod_item.searched")
    print(len(products))

    for product in products:
        try:
        # Navigate to the main information of each product to access the price list
            main_info = product.find_element(By.CSS_SELECTOR, ".prod_main_info")
            price_list = main_info.find_element(By.CSS_SELECTOR, ".prod_pricelist")

            # Extract the 'alt' text from the mall icon
            mall_icon_alt_text = price_list.find_element(By.CSS_SELECTOR, ".mall_icon img").get_attribute('alt')
            print("Mall Icon Alt Text:", mall_icon_alt_text)

            # Extract the price text
            price_text = price_list.find_element(By.CSS_SELECTOR, ".price_sect strong").text
            print("Price:", price_text)

            # Click on the link with class '.click_log_product_searched_price_'
            click_link = price_list.find_element(By.CSS_SELECTOR, ".click_log_product_searched_price_")
            driver.execute_script("arguments[0].click();", click_link)
            time.sleep(5)
        
            # Wait a bit for the action to complete or for the new page to load (if necessary)
                    # 현재 열려 있는 창/탭의 핸들을 모두 가져옴
            windows = driver.window_handles

            # 새 창으로 포커스를 전환 (가장 최근에 열린 창으로 가정)
            driver.switch_to.window(windows[-1])

            # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".")))  # 메인 윈도우가 재로드 됐다면, 대기

            time.sleep(5)
            # 새 창의 URL 가져오기
            current_url = driver.current_url
            print("New Window URL:", current_url)

            # 필요한 작업 수행 후, 창을 닫고 원래 창으로 돌아가기
            driver.close()

            time.sleep(1)

            driver.switch_to.window(main_window)  # 메인 윈도우로 전환
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))  # 메인 윈도우가 재로드 됐다면, 대기

            time.sleep(3)
        except StaleElementReferenceException:
            # 요소가 더 이상 유효하지 않을 때 예외 처리
            print("StaleElementReferenceException detected. Moving to the next product.")
            continue  # 다음 반복으로 넘어감
        

finally:
    # WebDriver 종료
    driver.quit()

KeyboardInterrupt: 

In [64]:
def extract_number_from_text(text):
    # 쉼표 제거
    no_comma = text.replace(",", "")
    # 원화 기호 및 문자 제거
    only_numbers = ''.join(filter(str.isdigit, no_comma))
    return int(only_numbers)

In [102]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def coupang_crawling(driver):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tab-titles")))  

    product_title = driver.find_element(By.CSS_SELECTOR, ".prod-buy-header__title").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".total-price strong").text.strip()  
    driver.find_element(By.NAME, "etc").click()

    time.sleep(random.uniform(0.1, 0.3))  # 로드 시간에 따라 조절 가능

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product-seller")))  # 새 창에서 콘텐츠가 로드될 때까지 대기
    partner_info_after_click = driver.find_element(By.CSS_SELECTOR, ".product-seller")
    partner_table = partner_info_after_click.find_element(By.CSS_SELECTOR, ".prod-delivery-return-policy-table tbody")
    partner_table_element = partner_table.find_elements(By.CSS_SELECTOR, "tr")

    if len(partner_table_element) <= 1:
        biz_name = driver.find_element(By.CSS_SELECTOR, ".product-seller table.prod-delivery-return-policy-table tbody > tr:nth-child(1) > td:nth-child(2)").text.strip()
        print(biz_name, "의 상품입니다")
        return []
    else:
        biz_name = driver.find_element(By.CSS_SELECTOR, ".product-seller table.prod-delivery-return-policy-table tbody > tr:nth-child(1) > td:nth-child(2)").text.strip()

# e-mail 정보 추출
        biz_email = driver.find_element(By.CSS_SELECTOR, ".product-seller table.prod-delivery-return-policy-table tbody > tr:nth-child(2) > td:nth-child(2)").text.strip()

# 연락처 정보 추출
        biz_phone = driver.find_element(By.CSS_SELECTOR, ".product-seller table.prod-delivery-return-policy-table tbody > tr:nth-child(2) > td:nth-child(4)").text.strip()
        current_url = driver.current_url
        print("New Window URL:", current_url)
        print(product_title, biz_name,biz_email, biz_phone)
        
        return [current_url, "쿠팡", product_title, biz_name, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&vendorItemId=70948736569&src=1032034&spec=10305199&addtag=400&ctag=1752199303&lptag=P1752199303&itime=20240326120408&pageType=PRODUCT&pageValue=1752199303&wPcid=17114222169871426060673&wRef=prod.danawa.com&wTime=20240326120408&redirect=landing&mcid=d8e678397b8e48888be45fa387a33eca&isAddedCart=")
        print(coupang_crawling(driver))
    finally:
        driver.quit()

New Window URL: https://www.coupang.com/vp/products/1752199303?itemId=2983891839&vendorItemId=70948736569&src=1032034&spec=10305199&addtag=400&ctag=1752199303&lptag=P1752199303&itime=20240326120408&pageType=PRODUCT&pageValue=1752199303&wPcid=17114222169871426060673&wRef=prod.danawa.com&wTime=20240326120408&redirect=landing&mcid=d8e678397b8e48888be45fa387a33eca&isAddedCart=
우양-무가당그린키위홀1kg (주)커피몰 / 오은이 coffeemall@ymail.com 1544-0807
['https://www.coupang.com/vp/products/1752199303?itemId=2983891839&vendorItemId=70948736569&src=1032034&spec=10305199&addtag=400&ctag=1752199303&lptag=P1752199303&itime=20240326120408&pageType=PRODUCT&pageValue=1752199303&wPcid=17114222169871426060673&wRef=prod.danawa.com&wTime=20240326120408&redirect=landing&mcid=d8e678397b8e48888be45fa387a33eca&isAddedCart=', '쿠팡', '우양-무가당그린키위홀1kg', '(주)커피몰 / 오은이', '1544-0807', 'coffeemall@ymail.com', 6100]


In [103]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def tmon_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".partner_info")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    # shop_content = driver.find_elements(By.CSS_SELECTOR, "body")

    pricing = driver.find_element(By.CSS_SELECTOR, ".deal_price_sell .number_unit").text.strip()
    partner_info = driver.find_element(By.CSS_SELECTOR, ".partner_info")
    print(partner_info)

    partner_button_click = driver.find_element(By.CSS_SELECTOR, ".partner_info button")
    driver.execute_script("arguments[0].click();", partner_button_click)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".partner_info .slide-ct")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    partner_info_after_click = driver.find_element(By.CSS_SELECTOR, ".partner_info .slide-ct")
    product_title = driver.find_element(By.CSS_SELECTOR, ".deal_title_main").text.strip()  

    biz_name = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(1) > td").text.strip()  
    biz_email = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(6) > td").text.strip()  
    biz_phone = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(7) > td").text.strip() 

    current_url = driver.current_url
        
    return [current_url, "티몬", product_title, biz_name, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("https://www.tmon.co.kr/deal/14431890826?opt_deal_srl=14487448682&coupon_srl=3383202&utm_source=danawa&utm_medium=affiliate&utm_term=205009_%EB%8B%A4%EB%82%98%EC%99%80DB&utm_content=&utm_campaign=%EB%8B%A4%EB%82%98%EC%99%80#tab=claim")
        print(tmon_crawling(driver))
    finally:
        driver.quit()

<selenium.webdriver.remote.webelement.WebElement (session="d7412dd2ef386937025b2ea8d3cf317f", element="f.07B6A874B2AEE1CF0B7B7C3A7B9D00A1.d.C49905C07DA62278BC8D9E9052CB6B05.e.160")>
['https://www.tmon.co.kr/deal/14431890826?opt_deal_srl=14487448682&coupon_srl=3383202&utm_source=danawa&utm_medium=affiliate&utm_term=205009_%EB%8B%A4%EB%82%98%EC%99%80DB&utm_content=&utm_campaign=%EB%8B%A4%EB%82%98%EC%99%80#tab=claim', '티몬', '제스프리 썬 골드키위/그린키위 특대과 외 사이즈별 골라담기', '주식회사 오렌지씨', '010-9177-2302\n티몬 고객센터 | 1544-6240', 'orangec1009@naver.com', 7900]


In [104]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def plus_wemakeprice_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#div_seller_detail_info_list")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    # shop_content = driver.find_elements(By.CSS_SELECTOR, "body")

    product_title = driver.find_element(By.CSS_SELECTOR, "#goods_name").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".price__num").get_attribute("data-price")
    
    seller_info = driver.find_element(By.CSS_SELECTOR, "#div_seller_detail_info_list")    

    biz_name = seller_info.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child(1) > td").text.strip()  
    biz_phone = seller_info.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child(2) > td").text.strip()
    biz_email = seller_info.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child(3) > td").text.strip()  
    

    current_url = driver.current_url
        
    return [current_url, "위메프플러스", product_title, biz_name, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("https://plus.wemakeprice.com/g/738942728?jaehuid=2026520850")
        print(plus_wemakeprice_crawling(driver))
    finally:
        driver.quit()

['https://plus.wemakeprice.com/g/738942728?jaehuid=2026520850', '위메프플러스', '[짱구샵]냉동 키위 1kg 연육제용 드레싱 껍질제거 그린키위 냉동과일', '짱구샵/김귀출', '+82-010-5864-2537', 'zzang9eshop@naver.com', 5500]


In [105]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def wemakeprice_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tab_info")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    product_title = driver.find_element(By.CSS_SELECTOR, ".deal_tit").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".sale_price .num").text.strip()  


    driver.find_element(By.CSS_SELECTOR, ".tab_info .on a").click()

    time.sleep(random.uniform(0.1, 0.3))  # 로드 시간에 따라 조절 가능

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".deal_shipinfo.box + .table_box")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    
    seller_info = driver.find_element(By.CSS_SELECTOR, ".deal_shipinfo.box + .table_box")    

    biz_name = seller_info.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child(2) > td").text.strip()  
    biz_phone = seller_info.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child(3) > td").text.strip()
    biz_email = seller_info.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child(4) > td").text.strip()  
    

    current_url = driver.current_url
        
    return [current_url, "위메프", product_title, biz_name, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("https://front.wemakeprice.com/product/2574079004?utm_source=danawa&utm_medium=PRICE_af&utm_campaign=null")
        print(wemakeprice_crawling(driver))
    finally:
        driver.quit()

['https://front.wemakeprice.com/product/2574079004?utm_source=danawa&utm_medium=PRICE_af&utm_campaign=null', '위메프', '맛있는 그린키위 랜덤과 1kg', '(주)멸치 / 남양우', '02-1588-9904', 'shopping@mel-chi.co.kr', 10910]


In [106]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def gmarket_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".vip-tabnavi.uxeposfix")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    product_title = driver.find_element(By.CSS_SELECTOR, ".itemtit").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".price_real").text.strip()  

    driver.find_element(By.CSS_SELECTOR, ".vip-tabnavi.uxeposfix ul li:nth-child(4) a").click()

    time.sleep(random.uniform(0.1, 0.3))  # 로드 시간에 따라 조절 가능

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".box__exchange-guide")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    
    seller_info = driver.find_element(By.CSS_SELECTOR, ".box__exchange-guide > div:nth-child(6)")    

    # 상호명 추출을 위한 CSS 선택자 수정 예시
    title = seller_info.find_element(By.CSS_SELECTOR, ".text__exchange-title").text.strip()
    biz_name = seller_info.find_element(By.CSS_SELECTOR, ".list__exchange-data .list-item:nth-child(1) .text__deco").text.strip()

    # 대표자명 추출을 위한 CSS 선택자 수정 예시
    ceo_name = seller_info.find_element(By.CSS_SELECTOR, ".list__exchange-data .list-item:nth-child(2) .text__deco").text.strip()
    biz_phone = seller_info.find_element(By.CSS_SELECTOR, ".list__exchange-data > li:nth-child(3) span").text.strip()
    biz_email = seller_info.find_element(By.CSS_SELECTOR, ".list__exchange-data > li:nth-child(7) span").text.strip()
    

    current_url = driver.current_url
        
    return [current_url, "G마켓", product_title, biz_name + "-" + ceo_name, biz_phone, biz_email, extract_number_from_text(pricing)]
    # return []
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        driver.get("https://item.gmarket.co.kr/Item?goodscode=3324949431&GoodsSale=Y&jaehuid=200002657")
        print(gmarket_crawling(driver))
    finally:
        driver.quit()

판매자 정보
['https://item.gmarket.co.kr/Item?goodscode=3324949431&GoodsSale=Y&jaehuid=200002657', 'G마켓', '새콤 달콤 국내산 참다래 그린 키위 1kg 중과(10~11과)', '건양유통-박우찬', '01038122831', 'hptomato@daum.net', 11280]


In [107]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def interpark_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".sellerArea")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    # shop_content = driver.find_elements(By.CSS_SELECTOR, "body")

    product_title = driver.find_element(By.CSS_SELECTOR, ".productName .subject").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".salePrice > em").text.strip()
    
    seller_info = driver.find_element(By.CSS_SELECTOR, ".sellerArea .infoContent")    

    biz_name = seller_info.find_element(By.CSS_SELECTOR, ".info:nth-child(1) > dd").text.strip()  
    biz_ceo = seller_info.find_element(By.CSS_SELECTOR, ".info:nth-child(2) > dd").text.strip()  
    biz_phone = seller_info.find_element(By.CSS_SELECTOR, ".info:nth-child(3) > dd").text.strip()
    biz_email = seller_info.find_element(By.CSS_SELECTOR, ".info:nth-child(6) > dd").text.strip()  
    

    current_url = driver.current_url
        
    return [current_url, "인터파크", product_title, biz_name + "/" + biz_ceo, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("https://shopping.interpark.com/product/productInfo.do?prdNo=13356562319&dispNo=016001&bizCd=P01415&utm_medium=affiliate&utm_source=danawa&utm_campaign=shop_20211015_danawa_p01415_cps&utm_content=conversion_47")
        print(interpark_crawling(driver))
    finally:
        driver.quit()

['https://shopping.interpark.com/product/productInfo.do?prdNo=13356562319&dispNo=016001&bizCd=P01415&utm_medium=affiliate&utm_source=danawa&utm_campaign=shop_20211015_danawa_p01415_cps&utm_content=conversion_47', '인터파크', '우양 무가당 그린키위 홀 1kg 냉동과일 빙수토핑 얼린 주스 주스 팥빙수재료', '부럽다마켓/대호유통상사/백종근', '070-8018-6362', 'bbjk686@naver.com', 11700]


In [108]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def auction_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".vip-tabcontent.vip-tabcontent_lt.tabview_exchange")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    # shop_content = driver.find_elements(By.CSS_SELECTOR, "body")

    product_title = driver.find_element(By.CSS_SELECTOR, ".itemtit").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".price_real").text.strip()
    

    driver.find_element(By.CSS_SELECTOR, "#tap_moving_4 > a").click()

    time.sleep(random.uniform(0.1, 0.3))  # 로드 시간에 따라 조절 가능

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".vip-tabcontent.vip-tabcontent_lt.tabview_exchange > .box__exchange-guide")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    seller_info = driver.find_element(By.CSS_SELECTOR, ".vip-tabcontent.vip-tabcontent_lt.tabview_exchange > .box__exchange-guide")    
    
    # biz_title = seller_info.find_element(By.CSS_SELECTOR, ".text__exchange-title").text.strip()  
    # print(biz_title)
    # print(seller_info.text.strip())

    biz_name = seller_info.find_element(By.CSS_SELECTOR, ".list__exchange-data .list-item:nth-child(1) > .text__deco").text.strip()  
    biz_ceo = seller_info.find_element(By.CSS_SELECTOR, ".list__exchange-data .list-item:nth-child(2) > .text__deco").text.strip()  
    biz_phone = seller_info.find_element(By.CSS_SELECTOR, ".list__exchange-data .list-item:nth-child(3) > .text__deco").text.strip()  
    biz_email = seller_info.find_element(By.CSS_SELECTOR, ".list__exchange-data .list-item:nth-child(7) > .text__deco").text.strip()  
    

    current_url = driver.current_url
        
    return [current_url, "옥션", product_title, biz_name + "/" + biz_ceo, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("http://itempage3.auction.co.kr/DetailView.aspx?ItemNo=E268939549&frm3=V2")
        print(auction_crawling(driver))
    finally:
        driver.quit()

['http://itempage3.auction.co.kr/DetailView.aspx?ItemNo=E268939549&frm3=V2', '옥션', '참좋은과일 그리스 그린키위 소과 과당 90g 내외 1kg', '쏠켓/박종택', '010-2841-6643', '24solket@naver.com', 15120]


In [127]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def lotte_on_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".scrollTabWrapper li:nth-child(4) > a")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    # shop_content = driver.find_elements(By.CSS_SELECTOR, "body")

    product_title = driver.find_element(By.CSS_SELECTOR, ".pd-widget1__product-name").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".pd-price__info--number").text.strip()

    driver.find_element(By.CSS_SELECTOR, ".scrollTabWrapper li:nth-child(4) > a").click()

    time.sleep(random.uniform(0.1, 0.3))  # 로드 시간에 따라 조절 가능

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".detailContents.deliveryInfo .productCollapseContents:nth-child(1) .guideContents:nth-child(1)")))  # 새 창에서 콘텐츠가 로드될 때까지 대기


    seller_info = driver.find_element(By.CSS_SELECTOR, ".detailContents.deliveryInfo .productCollapseContents:nth-child(1) .guideContents:nth-child(1)")    
    
    print(seller_info.text.strip())

    # biz_name = seller_info.find_element(By.CSS_SELECTOR, "tr:nth-child(3) > td:nth-child(1)").text.strip()  
    # biz_ceo = seller_info.find_element(By.CSS_SELECTOR, "tr:nth-child(4) > td:nth-child(1)").text.strip()  
    # biz_phone = seller_info.find_element(By.CSS_SELECTOR, "tr:nth-child(8) > td:nth-child(1)").text.strip()  
    # biz_email = seller_info.find_element(By.CSS_SELECTOR, "tr:nth-child(9) > td:nth-child(1)").text.strip()  
    

    # current_url = driver.current_url
        
    # return [current_url, "롯데ON", product_title, biz_name + "/" + biz_ceo, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("https://www.lotteon.com/p/product/LO2220890750?sitmNo=LO2220890750_2220890751&ch_no=100071&ch_dtl_no=1000227&entryPoint=pcs&dp_infw_cd=CHT")
        print(lotte_on_crawling(driver))
    finally:
        driver.quit()

ElementClickInterceptedException: Message: element click intercepted: Element <a>...</a> is not clickable at point (790, 838). Other element would receive the click: <div class="v--modal-background-click">...</div>
  (Session info: chrome=123.0.6312.59)
Stacktrace:
0   chromedriver                        0x00000001007e43e8 chromedriver + 4326376
1   chromedriver                        0x00000001007dc8b0 chromedriver + 4294832
2   chromedriver                        0x0000000100408088 chromedriver + 278664
3   chromedriver                        0x0000000100450554 chromedriver + 574804
4   chromedriver                        0x000000010044ead8 chromedriver + 568024
5   chromedriver                        0x000000010044cc58 chromedriver + 560216
6   chromedriver                        0x000000010044c188 chromedriver + 557448
7   chromedriver                        0x00000001004413ac chromedriver + 512940
8   chromedriver                        0x0000000100440dd0 chromedriver + 511440
9  

In [130]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def street11_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tabpanelDetail4")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    seller_info = driver.find_element(By.CSS_SELECTOR, "#tabpanelDetail4 .prodc_return_wrap .prdc_detail_table:nth-child(9)")    
    product_title = driver.find_element(By.CSS_SELECTOR, ".c_product_info_title.c_product_info_title_coupon .title").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".price_block .price .value").text.strip()
    

    biz_name = seller_info.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(1) td:nth-child(2)").text.strip()  
    biz_ceo = seller_info.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(1) td:nth-child(4)").text.strip()
    biz_phone = seller_info.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(2) td:nth-child(4)").text.strip()
    biz_email = seller_info.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(4) td:nth-child(2)").text.strip()
    

    current_url = driver.current_url
        
    return [current_url, "11번가", product_title, biz_name + "/" + biz_ceo, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("https://www.11st.co.kr/products/6620952725?utm_term=&utm_campaign=%B4%D9%B3%AA%BF%CDpc_%B0%A1%B0%DD%BA%F1%B1%B3%B1%E2%BA%BB&utm_source=%B4%D9%B3%AA%BF%CD_PC_PCS&utm_medium=%B0%A1%B0%DD%BA%F1%B1%B3")
        print(street11_crawling(driver))
    finally:
        driver.quit()

['https://www.11st.co.kr/products/6620952725?utm_term=&utm_campaign=%B4%D9%B3%AA%BF%CDpc_%B0%A1%B0%DD%BA%F1%B1%B3%B1%E2%BA%BB&utm_source=%B4%D9%B3%AA%BF%CD_PC_PCS&utm_medium=%B0%A1%B0%DD%BA%F1%B1%B3', '옥션', '제주도 그린키위 참다래 1kg_MC', '멸치쇼핑/주식회사 멸치 │ 남양우', '1588-9904', 'shopping@mel-chi.co.kr', 8300]


In [132]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote
import time 
import random




def lotte_home_crawling(driver):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".division_product_tab")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    driver.find_element(By.CSS_SELECTOR, ".division_product_tab .tab2 > a").click()

    time.sleep(random.uniform(0.1, 0.3))  # 로드 시간에 따라 조절 가능

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".wrap_detail.content2 .detail .row_info:nth-child(3) .table_1")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

    seller_info = driver.find_element(By.CSS_SELECTOR, ".wrap_detail.content2 .detail .row_info:nth-child(3) .table_1")    
    product_title = driver.find_element(By.CSS_SELECTOR, ".title_product .tit").text.strip()  
    pricing = driver.find_element(By.CSS_SELECTOR, ".price_product.price_black .num").text.strip()
    

    biz_name = seller_info.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(13) td:nth-child(2)").text.strip()  
    biz_phone = seller_info.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(15) td:nth-child(2)").text.strip()
    biz_email = ""
    

    current_url = driver.current_url
        
    return [current_url, "롯데홈쇼핑", product_title, biz_name, biz_phone, biz_email, extract_number_from_text(pricing)]
    

if __name__ == "__main__":
    try: 
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(options=options)
        # driver = webdriver.Safari()
        # driver.get("https://www.coupang.com/vp/products/1752199303?itemId=2983891839&isAddedCart=")
        driver.get("https://www.lotteimall.com/goods/viewGoodsDetail.lotte?goods_no=2534307366&chl_dtl_no=&chl_no=23")
        print(lotte_home_crawling(driver))
    finally:
        driver.quit()

['https://www.lotteimall.com/goods/viewGoodsDetail.lotte?goods_no=2534307366&chl_dtl_no=&chl_no=23', '롯데홈쇼핑', '산지직송 제주 참다래 그린키위 1kg (10-13과 내외)', '(주)규(GYU Co., Ltd.)', '010-8875-6704', '', 15400]


In [133]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, quote


def crawlingData(page, search_keyword):
    # Safari WebDriver 인스턴스 생성
    # driver = webdriver.Safari()
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(options=options)
    craling_list = []

    try:
        # 페이지 로드
        url = f'https://search.danawa.com/dsearch.php?query={search_keyword}&checkedInfo=N&volumeType=allvs&page={page}&limit=40&sort=priceASC&list=list&boost=true&tab=main&addDelivery=Y&coupangMemberSort=N&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=16325265&defaultPhysicsCategoryCode=46803%7C46818%7C50379%7C0&defaultVmTab=10&defaultVaTab=389&isZeroPrice=Y&quickProductYN=N&priceUnitSort=Y&priceUnitSortOrder=A'
        driver.get(url)
        # driver.get(f'https://search.danawa.com/dsearch.php?query={search_keyword}&checkedInfo=N&volumeType=allvs&page={page}&limit=40&sort=priceASC&list=list&boost=true&tab=goods&addDelivery=Y&coupangMemberSort=N&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=16325265&defaultPhysicsCategoryCode=46803%7C46818%7C50379%7C0&defaultVmTab=10&defaultVaTab=387&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A')

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))
        main_window = driver.current_window_handle  # 메인 윈도우 핸들 저장
        
        products = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product_list .prod_item.searched")))

        print(len(products))

        for index, product in enumerate(products[:5]):
            try:
                # 새로운 요소 참조를 얻기 위해 매번 elements를 다시 조회합니다.
                product = driver.find_elements(By.CSS_SELECTOR, ".product_list .prod_item.searched")[index]
                main_info = product.find_element(By.CSS_SELECTOR, ".prod_main_info")
                price_list = main_info.find_element(By.CSS_SELECTOR, ".prod_pricelist")

                mall_icon_alt_text = price_list.find_element(By.CSS_SELECTOR, ".mall_icon img").get_attribute('alt').strip()
                print("Mall Icon Alt Text:", mall_icon_alt_text)

                price_text = price_list.find_element(By.CSS_SELECTOR, ".price_sect strong").text
                print("Price:", price_text)

                # 새 창에서 작업
                click_link = price_list.find_element(By.CSS_SELECTOR, ".click_log_product_searched_price_")
                driver.execute_script("arguments[0].click();", click_link)
                
                # 새 창이 나타날 때까지 기다립니다.
                WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                windows = driver.window_handles
                new_window = [window for window in windows if window != main_window][0]
                driver.switch_to.window(new_window)
                
                # # 새 창에서 필요한 정보를 가져온 후 닫기
                if mall_icon_alt_text == "티몬":


                    # WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".partner_info")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

                    # # shop_content = driver.find_elements(By.CSS_SELECTOR, "body")

                    # partner_info = driver.find_element(By.CSS_SELECTOR, ".partner_info")
                    # print(partner_info)

                    # partner_button_click = driver.find_element(By.CSS_SELECTOR, ".partner_info button")
                    # driver.execute_script("arguments[0].click();", partner_button_click)

                    # WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".partner_info .slide-ct")))  # 새 창에서 콘텐츠가 로드될 때까지 대기

                    # partner_info_after_click = driver.find_element(By.CSS_SELECTOR, ".partner_info .slide-ct")
                    # product_title = driver.find_element(By.CSS_SELECTOR, ".deal_title_main").text.strip()  

                    # biz_name = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(1) > td").text.strip()  
                    # biz_email = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(6) > td").text.strip()  
                    # biz_phone = partner_info_after_click.find_element(By.CSS_SELECTOR, "tr:nth-child(7) > td").text.strip()  

                    tmon_list = tmon_crawling(driver)

                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    # print(product_title, biz_name,biz_email, biz_phone)
                    
                    craling_list.append(tmon_list)

                elif mall_icon_alt_text == "쿠팡":
                    # WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".prod-atf-main")))  
                    coupang_list = coupang_crawling(driver)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    if len(coupang_list) == 0:
                        print("회원정보 없음")
                    else:
                        current_url = driver.current_url
                        coupang_list.insert(0, current_url)
                        
                        craling_list.append(coupang_list)
                elif mall_icon_alt_text == "위메프플러스":
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".goods_wrap")))  
                    plus_we_make_market_list = plus_wemakeprice_crawling(driver)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    craling_list.append(plus_we_make_market_list)
                elif mall_icon_alt_text == "위메프":
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".container")))  
                    we_make_market_list = wemakeprice_crawling(driver)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    craling_list.append(we_make_market_list)
                elif mall_icon_alt_text == "G마켓":
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".item-topinfowrap")))  
                    gmarket_list = gmarket_crawling(driver)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    craling_list.append(gmarket_list)
                elif mall_icon_alt_text == "인터파크":
                    prev_popup_window = driver.current_window_handle  # 메인 윈도우 핸들 저장

                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".productInfomation")))  
                    windows = driver.window_handles
                    new_window = next(window for window in windows if window != prev_popup_window and window != main_window)
                    driver.switch_to.window(new_window)
                    driver.close()
                    driver.switch_to.window(prev_popup_window)
                    interpark_list = interpark_crawling(driver)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    crawling_list.append(interpark_list)
                elif mall_icon_alt_text == "옥션":
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".vip-content")))  
                    auction_list = auction_crawling(driver)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    crawling_list.append(auction_list)
                elif mall_icon_alt_text == "롯데ON":
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".productWrap")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "11번가":
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".l_product_summary")))  
                    street11_list =street11_crawling(driver)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                    crawling_list.append(street11_list)
                elif mall_icon_alt_text == "이마트인터넷쇼핑몰":
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".cdtl_row_top")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "SSG.COM":
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".content_primary")))  
                    current_url = driver.current_url
                    print("New Window URL:", current_url)
                elif mall_icon_alt_text == "롯데홈쇼핑":
                    prev_popup_window = driver.current_window_handle  # 메인 윈도우 핸들 저장

                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".detail_sec")))  
                    windows = driver.window_handles
                    new_window = next(window for window in windows if window != prev_popup_window and window != main_window)
                    driver.switch_to.window(new_window)
                    driver.close()
                    driver.switch_to.window(prev_popup_window)

                    lotte_home_list = lotte_home_crawling(driver)
                    crawling_list.append(lotte_home_list)
                    current_url = driver.current_url
                    print("New Window URL:", current_url)


                
                
                

                driver.close()
                driver.switch_to.window(main_window)  # 메인 윈도우로 전환

                # 원래 윈도우로 돌아온 후 필요한 웹 요소를 다시 찾습니다.
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))


            except StaleElementReferenceException:
                continue

    except StaleElementReferenceException:
        print(StaleElementReferenceException)

    finally:
        driver.quit()
        return craling_list

list = []

if __name__ == "__main__":
    crawling_list = crawlingData(1, "그린키위 1kg")
    print(crawling_list)
    list = crawling_list


36
Mall Icon Alt Text: 티몬
Price: 6,960
<selenium.webdriver.remote.webelement.WebElement (session="8faff785680fcabf148e9b9ecb45a9a1", element="f.E4AB0CF835E2E6B5C29B16B11A0D7FDC.d.4F856F1EEE4C8CE7CAF984AD5125AF5D.e.65")>
New Window URL: https://www.tmon.co.kr/deal/14431890826?opt_deal_srl=14487448682&coupon_srl=3383202&utm_source=danawa&utm_medium=affiliate&utm_term=205009_%EB%8B%A4%EB%82%98%EC%99%80DB&utm_content=&utm_campaign=%EB%8B%A4%EB%82%98%EC%99%80
Mall Icon Alt Text: 쿠팡
Price: 7,210
쿠팡 1577-7011 의 상품입니다
New Window URL: https://www.coupang.com/vp/products/5750079900?itemId=15060092714&vendorItemId=82282468174&src=1032034&spec=10305197&addtag=400&ctag=5750079900&lptag=I15060092714&itime=20240326175031&pageType=PRODUCT&pageValue=5750079900&wPcid=17114430312196883038128&wRef=prod.danawa.com&wTime=20240326175031&redirect=landing&mcid=968645ba66c14a14ab0b360cdce9a195&isAddedCart=
회원정보 없음
Mall Icon Alt Text: 쿠팡
Price: 7,680
[['https://www.tmon.co.kr/deal/14431890826?opt_deal_srl=144874

36
Mall Icon Alt Text: 티몬
Price: 6,960


NoSuchElementException: Message: ; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import time


def wait_for_page_load(driver, timeout=30):
    old_page = driver.find_element(By.TAG_NAME, 'html')
    yield
    WebDriverWait(driver, timeout).until(
        EC.staleness_of(old_page)
    )

def check_page_is_loaded(driver, timeout=30):
    # 페이지가 완전히 로드될 때까지 기다림 (document.readyState가 complete가 될 때까지)
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )

# Safari WebDriver 인스턴스 생성
driver = webdriver.Safari()

try:
    # 페이지 로드
    # driver.get('https://prod.danawa.com/bridge/go_link_goods.php?cate_c1=46803&cate_c2=46818&cate_c3=50379&cate_c4=0&cmpny_c=TP40F&link_prod_c=I15060092714&keyword=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&r=17113687248228&cate=16325265')
    driver.get('https://prod.danawa.com/bridge/go_link_goods.php?cate_c1=46803&amp;cate_c2=46818&amp;cate_c3=50379&amp;cate_c4=0&amp;cmpny_c=TP40F&amp;link_prod_c=I15060092714&amp;keyword=%EA%B7%B8%EB%A6%B0%ED%82%A4%EC%9C%84+1kg&amp;r=17113687248228&amp;cate=16325265')
               

    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product_list")))
    # main_window = driver.current_window_handle  # 메인 윈도우 핸들 저장
    
    # products = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product_list .prod_item.searched")))

    # print(len(products))

    # for index, product in enumerate(products):
    time.sleep(10)
       
finally:
    driver.quit()





In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [ ]:
def append_crawling_data(service, spreadsheet_id, values, sheet_name):
    range_name = f'{sheet_name}!b2:D'


    body = {
    'values': values    
    }

    # 스프레드시트에 데이터 쓰기
    result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, range=range_name,
    valueInputOption='RAW', body=body).execute()

    print(result)

In [ ]:
# unit_pricing_spreadsheet_id = "1oxak1ZgQi3YhbC5UKPcmlrjV9ekMt53BVkHlycBW3uQ"
#크롤링 데이터 원본
unit_pricing_spreadsheet_id = "132EosVTtzTuqNrfJ2mDxRD-TbOHxDGdGXKwpgA92y54"
worksheet_name = '캄므커머스_데이터원본'
arranged_worksheet_name = '캄므커머스'

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    append_crawling_data(service_sheets, unit_pricing_spreadsheet_id,product_processed_data, worksheet_name)



{'spreadsheetId': '132EosVTtzTuqNrfJ2mDxRD-TbOHxDGdGXKwpgA92y54', 'updatedRange': "'캄므커머스_데이터원본'!B2:D220", 'updatedRows': 219, 'updatedColumns': 3, 'updatedCells': 657}
